## accesso al daily firstrate

In [1]:
from revelation.data.enums import DataProvider
from revelation.data.instruments import FuturesContract
from revelation.data.loading import (
    Catalog,
)
import pandas as pd

# from revelation.visualization.visualization import hvplot_ohlc
from revelation.data.symbol import Symbol

catalog = Catalog()
symbols = [Symbol(f"E6-{m}-{y}") for m in "HMUZ" for y in range(2020, 2025)]
contracts: list[FuturesContract] = catalog.get_futures_contracts(
    symbols, DataProvider.FIRSTRATE, ["1day", "1min"], is_raw_data=True
)
print(*contracts, sep="\n")

Futures contract: E6-H-2020
Futures contract: E6-M-2020
Futures contract: E6-U-2020
Futures contract: E6-Z-2020
Futures contract: E6-H-2021
Futures contract: E6-M-2021
Futures contract: E6-U-2021
Futures contract: E6-Z-2021
Futures contract: E6-H-2022
Futures contract: E6-M-2022
Futures contract: E6-U-2022
Futures contract: E6-Z-2022
Futures contract: E6-H-2023
Futures contract: E6-M-2023
Futures contract: E6-U-2023
Futures contract: E6-Z-2023
Futures contract: E6-H-2024
Futures contract: E6-M-2024
Futures contract: E6-U-2024
Futures contract: E6-Z-2024


L'intersezione fra gli indici di curr e next è uguale all'indice ottenuto tramite il join "inner"

In [2]:
from revelation.data.enums import RolloverRule
from revelation.data.instruments import ContinuousFuturesContract


continuous = ContinuousFuturesContract.from_individuals(contracts, RolloverRule.EXPIRY)
index = -1
print(
    contracts[index],
    f"expiry {contracts[index].expiration}, {contracts[index].expiration.day_name()}",
)
continuous.market_data["D"]

Futures contract: E6-Z-2024 expiry 2024-12-16 00:00:00-05:00, Monday


,open,high,low,close,volume,open_interest,symbol
timestamp,,,,,,,
2018-10-02 00:00:00-04:00,1.21100,1.21110,1.21000,1.21110,260,129,E6-H-2020
2018-10-03 00:00:00-04:00,1.21030,1.21030,1.20840,1.20840,76,165,E6-H-2020
2018-10-04 00:00:00-04:00,1.20900,1.20900,1.20810,1.20810,40,167,E6-H-2020
2018-10-23 00:00:00-04:00,1.20195,1.20195,1.20195,1.20195,10,177,E6-H-2020
2018-10-24 00:00:00-04:00,1.19340,1.19340,1.19340,1.19340,20,197,E6-H-2020
...,...,...,...,...,...,...,...
2024-12-10 00:00:00-05:00,1.05590,1.05705,1.05010,1.05290,404294,446689,E6-Z-2024
2024-12-11 00:00:00-05:00,1.05290,1.05415,1.04820,1.04935,447931,296731,E6-Z-2024
2024-12-12 00:00:00-05:00,1.04990,1.05315,1.04645,1.04735,376701,190809,E6-Z-2024


In [ ]:
from revelation.data.enums import RolloverRule
from revelation.data.instruments import ContinuousFuturesContract


continuous = ContinuousFuturesContract.from_individuals(
    contracts, RolloverRule.OPEN_INTEREST
)
continuous.market_data["D"].index.is_

True

In [9]:
from revelation.visualization.visualization import hvplot_ohlc

hvplot_ohlc(continuous.market_data["D"], "continuous")

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


:Overlay
   .Segments.I   :Segments   [timestamp,low,timestamp,high]   (open,high,low,close,lbound,ubound)
   .Rectangles.I :Rectangles   [lbound,open,ubound,close]   (open,high,low,close,lbound,ubound)

In [56]:
from revelation.data.enums import DayOfWeek
from revelation.data.aggregation import subsample_ohlc


ohlc_sub = subsample_ohlc(ohlc, timeframe="1h")
# solo dal 2024
ohlc_sub = ohlc_sub.dropna(subset=["open", "high", "low", "close"], how="all")["2023":]

for day in DayOfWeek:

    mondays: pd.DataFrame = ohlc_sub[ohlc_sub.index.dayofweek == day]
    daily_range = (
        mondays["high"].resample("D").max() - mondays["low"].resample("D").min()
    )
    print(day.name, round(daily_range.dropna().mean(), 4))

MON 0.0073
TUE 0.0095
WED 0.0082
THU 0.0089
FRI 0.0096
SAT nan
SUN 0.0033


## accesso ai dati tradingview

In [5]:
import vectorbtpro as vbt
from revelation.time_utils import STANDARD_TIMEZONE
from dotenv import load_dotenv

load_dotenv()
import os

# tradingview_client = vbt.TVData.resolve_client(
#     # username=os.getenv("TRADINGVIEW_USERNAME"),
#     # password=os.getenv("TRADINGVIEW_PASSWORD"),
#     auth_token=os.getenv("TRADINGVIEW_AUTH_TOKEN"),
# )

In [6]:
data = vbt.TVData.fetch_symbol(
    "CME:6EH2024",
    timeframe="1 hour",  # client=tradingview_client
)
data[0]

,Open,High,Low,Close,Volume
datetime,,,,,
2021-02-05 20:00:00+00:00,1.24310,1.24310,1.24310,1.24310,1.0
2021-02-08 20:00:00+00:00,1.24470,1.24470,1.24470,1.24470,1.0
2021-02-09 20:00:00+00:00,1.25070,1.25070,1.25070,1.25070,1.0
2021-02-10 20:00:00+00:00,1.25110,1.25110,1.25110,1.25110,1.0
2021-02-11 20:00:00+00:00,1.25160,1.25160,1.25160,1.25160,1.0
...,...,...,...,...,...
2024-03-18 10:00:00+00:00,1.08995,1.09010,1.08970,1.09000,52.0
2024-03-18 11:00:00+00:00,1.09015,1.09045,1.09015,1.09035,103.0
2024-03-18 12:00:00+00:00,1.09025,1.09045,1.08915,1.08915,101.0


In [7]:
df: pd.DataFrame = data[0].copy()
df.index = df.index.tz_convert(STANDARD_TIMEZONE)
df.columns = df.columns.str.lower()
df

,open,high,low,close,volume
datetime,,,,,
2021-02-05 15:00:00-05:00,1.24310,1.24310,1.24310,1.24310,1.0
2021-02-08 15:00:00-05:00,1.24470,1.24470,1.24470,1.24470,1.0
2021-02-09 15:00:00-05:00,1.25070,1.25070,1.25070,1.25070,1.0
2021-02-10 15:00:00-05:00,1.25110,1.25110,1.25110,1.25110,1.0
2021-02-11 15:00:00-05:00,1.25160,1.25160,1.25160,1.25160,1.0
...,...,...,...,...,...
2024-03-18 06:00:00-04:00,1.08995,1.09010,1.08970,1.09000,52.0
2024-03-18 07:00:00-04:00,1.09015,1.09045,1.09015,1.09035,103.0
2024-03-18 08:00:00-04:00,1.09025,1.09045,1.08915,1.08915,101.0


In [8]:
import json
import uuid
from typing import Any

import pandas as pd
from IPython.display import HTML, display, Markdown

__all__ = ["plot"]


# TODO un to_json probabilmente è più efficiente
def _to_lw_format(df: pd.DataFrame) -> list[dict[str, Any]]:
    """Convert OHLC DataFrame to list-of-dicts format expected by lightweight-charts."""
    out = [
        {
            "time": int(pd.Timestamp(ts).timestamp()),  # seconds since Epoch
            "open": float(row["open"]),
            "high": float(row["high"]),
            "low": float(row["low"]),
            "close": float(row["close"]),
        }
        for ts, row in df.iterrows()
    ]
    return out


def plot(
    dataframe: pd.DataFrame,
    *,
    height: int = 400,
    width: str | int = "100%",
) -> None:
    # --- validation ---------------------------------------------------------
    required = {"open", "high", "low", "close"}
    if not required.issubset(dataframe.columns):
        raise ValueError(f"DataFrame must contain columns: {required}")
    if not isinstance(dataframe.index, pd.DatetimeIndex):
        raise TypeError("DataFrame index must be a DateTimeIndex")

    # --- prepare data & ids --------------------------------------------------
    candles_json = json.dumps(_to_lw_format(dataframe), separators=(",", ":"))
    div_id = f"lwch_{uuid.uuid4().hex}"

    # --- HTML/JS payload -----------------------------------------------------
    html = f"""
    <div id="{div_id}" style="width:{width};height:{height}px;"></div>
    <script type="text/javascript">
    // Helper to load a script only once
    function loadScript(src) {{
        return new Promise((resolve, reject) => {{
            if (document.querySelector(`script[src="${{src}}"]`)) {{ resolve(); return; }}
            const s = document.createElement('script');
            s.src = src;
            s.onload = resolve;
            s.onerror = reject;
            document.head.appendChild(s);
        }});
    }}

    (async () => {{
        // Load dependencies (Vue 3 + Lightweight-Charts)
        await loadScript('https://unpkg.com/vue@3/dist/vue.global.prod.js');
        await loadScript('https://unpkg.com/lightweight-charts@4.1.0/dist/lightweight-charts.standalone.production.js');

        const {{ createApp, onMounted, ref }} = Vue;

        const App = {{
            setup() {{
                const container = ref(null);

                onMounted(() => {{
                    const chart = LightweightCharts.createChart(container.value, {{
                        width: container.value.clientWidth,
                        height: {height},
                        layout: {{
                            background: {{ type: 'solid', color: '#ffffff' }},
                            textColor: '#333',
                        }},
                        grid: {{
                            vertLines: {{ color: '#f0f3fa' }},
                            horzLines: {{ color: '#f0f3fa' }},
                        }},
                    }});
                    const series = chart.addCandlestickSeries();
                    series.setData({candles_json});

                    // Keep chart responsive
                    new ResizeObserver(entries => {{
                        for (const entry of entries) {{
                            chart.applyOptions({{ width: entry.contentRect.width }});
                        }}
                    }}).observe(container.value);
                }});

                return {{ container }};
            }},
            template: '<div ref="container" style="width:100%;height:100%;"></div>',
        }};

        createApp(App).mount('#{div_id}');
    }})();
    </script>
    """

    display(HTML(html))

voglio prendere un df, inserirlo in una funzione plot e questa deve comunicare con vue tv
cosi da aprire un grafico immediatamente, non mi interessa se direttamente con jupyter output o webpage.

il df deve essere in memory

prova disegno box, orari di tv chart sono comunque errati

In [9]:
# BUG gli orari sono comunque sbagliati
chart = set_chart(df)
chart.box(
    df.index[-3],
    df["low"].iloc[-3],
    df.index[-1],
    df["high"].iloc[-1],
)
chart.load()

NameError: name 'set_chart' is not defined